In [264]:
# Source: https://github.com/PradipKumarChaudhary1/N-gram-model
import numpy as np

In [265]:
first_possible_word = {}
second_possible_word = {}
transition = {}


def expandDict(
    dictionary, key, value
):  # storing into dictionary current word as a 'value' and previous word as a 'key'
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)


def get_next_probability(word_list):  # finding probability of each word
    word_list_length = len(word_list)
    probability_dict = {}
    for item in word_list:
        probability_dict[item] = (
            probability_dict.get(item, 0) + 1
        )  # calculating frequncy of any word
    for (
        key,
        value,
    ) in probability_dict.items():  # calculating probability and store into dictionary according to thire key and probability as a value
        probability_dict[key] = (value + 1) / (word_list_length + 9484)
    probability_dict = sort_prob(probability_dict)
    return probability_dict


def sort_prob(dictionary):  # sorting of dictionary through values
    keys_list = list(dictionary.keys())
    values_list = list(dictionary.values())
    for i in range(len(values_list) - 1):
        for j in range(len(values_list) - i - 1):
            if values_list[j] < values_list[j + 1]:
                temp = values_list[j]
                values_list[j] = values_list[j + 1]
                values_list[j + 1] = temp
                temp = keys_list[j]
                keys_list[j] = keys_list[j + 1]
                keys_list[j + 1] = temp

    sort_dict = {}
    for i in range(len(values_list)):
        sort_dict[keys_list[i]] = values_list[i]
    return sort_dict


def trainModel(filepath):
    for line in open(filepath):
        tokens = line.rstrip().lower().split()
        tokens_length = len(tokens)
        for i in range(tokens_length):
            token = tokens[i]
            if i == 0:  # if word is ffirst word of every sentence
                first_possible_word[token] = first_possible_word.get(token, 0) + 1
            else:
                prev_token = tokens[i - 1]
                if i == 1:  # if word is 2nd word of the senetnce
                    expandDict(second_possible_word, prev_token, token)
                if i == tokens_length - 1:  # if word is last of sentence
                    expandDict(transition, (prev_token, token), "END")
                else:
                    prev_prev_token = tokens[i - 2]
                    expandDict(transition, (prev_prev_token, prev_token), token)

    first_possible_word_total = sum(
        first_possible_word.values()
    )  # finding total frequency n first_possible word
    for (
        key,
        value,
    ) in first_possible_word.items():  # calculating probability of first word in each sentence and store according to that word
        first_possible_word[key] = (value + 1) / (first_possible_word_total + 9484)

    for prev_word, next_word_list in second_possible_word.items():
        second_possible_word[prev_word] = get_next_probability(next_word_list)

    for word_pair, next_word_list in transition.items():
        transition[word_pair] = get_next_probability(next_word_list)


def next_word_one(tpl):
    d = second_possible_word.get(
        tpl
    )  # tpl match with key and return value, value is already dictionary
    if d is not None:
        return list(d.keys())[:5]  # since d is already dictionary so return keys
    else:
        probs = np.array(list(first_possible_word.values()))
        probs = probs / np.sum(probs) 
        return np.random.choice(a=list(first_possible_word.keys()), size=5, p=probs).tolist()

def next_word(tpl):
    if type(tpl) is str:  # it is first word of string.. return from second word
        return next_word_one(tpl.lower())
    if (
        type(tpl) is tuple
    ):  # incoming words are combination of two words.. find next word now based on transitions
        d = transition.get(tpl)
        if d is None:
            return next_word_one(tpl[1])
        return list(d.keys())[:5]
    return None  # wrong input.. return nothing

In [266]:
trainModel("./test_nextword.txt")

In [267]:
currentWord = "We"

wordList = [currentWord]
for i in range(100):
    if len(wordList) == 1:
        res = next_word(wordList[0])
    else:
        res = next_word((wordList[-2], wordList[-1]))
    wordList.append(res[0])

print(" ".join(wordList))

We have you not find me ungrateful holmes turned to the other END but i am sure that i have no doubt that the doctor s advice and help that is the first place we may take it that it was a very serious one to the other END under these circumstances the young lady s purse and watch if it were in the morning END remarkable as being a man who is in a few minutes with his head sunk upon his face END as i have no doubt that the doctor s advice and help that is the first
